In [20]:
# IMPORTING LIBRARIES

import matplotlib
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import seaborn as sb
import scipy
import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import transforms
from sklearn.model_selection import train_test_split

In [21]:
# CONFIGURING SETTINGS

pd.set_option('display.max_rows', 60, 'display.max_columns', None)
#np.set_printoptions(precision=3)
#sb.set(style='darkgrid')


In [22]:
# LOADING DATASETS

datasets = ['datasets/Monday-WorkingHours.pcap_ISCX.csv','datasets/Tuesday-WorkingHours.pcap_ISCX.csv',
            'datasets/Wednesday-WorkingHours.pcap_ISCX.csv','datasets/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
            'datasets/Thursday-WorkingHours-Afternoon-Infiltration.pcap_ISCX.csv','datasets/Friday-WorkingHours-Morning.pcap_ISCX.csv',
            'datasets/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv','datasets/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv']

d1 = pd.read_csv(datasets[0])
d2 = pd.read_csv(datasets[1])
d3 = pd.read_csv(datasets[2])
d4 = pd.read_csv(datasets[3])
d5 = pd.read_csv(datasets[4])
d6 = pd.read_csv(datasets[5])
d7 = pd.read_csv(datasets[6])
d8 = pd.read_csv(datasets[7])

frames = [d1, d2, d3, d4, d5, d6, d7, d8]
main_df = pd.concat(frames)

In [23]:
# REPLACING LABEL VALUES

data = main_df.drop(columns=[' Label'])
y = main_df[' Label'].replace(['BENIGN', 'PortScan', 'FTP-Patator', 'SSH-Patator', 'Web Attack � Brute Force', 'Bot', 
                               'Infiltration', 'Heartbleed', 'Web Attack � XSS', 'Web Attack � Sql Injection', 'DoS slowloris', 
                               'DoS Slowhttptest', 'DoS Hulk', 'DoS GoldenEye', 'DDoS' ], 
                               [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14])
data[' Label'] = y
data.shape

(2830743, 79)

In [24]:
# SPLIT TEST AND TRAIN DATA
learning_rate = 0.001
batch_size = 64
num_epochs = 1000

train, test = train_test_split(data, test_size=0.2)

train_loader = DataLoader(dataset=train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test, batch_size=batch_size, shuffle=True)

# SET DEVICE
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [25]:
# CREATE NEURAL NETWORK
class DetectionModel(nn.Module):
    def __init__(self, input_size, hidden1_size, hidden2_size, num_classes):
        super(DetectionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden2_size, num_classes)
        
    def foward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

In [26]:
# INITIALIZE NETWORK
model = DetectionModel(79, 100, 50, 15)

# LOSS AND OPTIMIZER
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [36]:
# TRAIN NETWORK
for epoch in range(num_epochs):
    for data, targets in train_loader:
        # ATTEMPT GETTING DATA TO CUDA
        data = data.to(device=device)
        targets = targets.to(device=device)

        # GET TO CORRECT SHAPE
        data = data.reshape(data.shape[0], -1)

        # FOWARD
        scores = model(data)
        loss = criterion(scores, targets)

        # BACKWARD
        optimizer.sero_grad()
        loss.backward()

        # GRADIENT DESCENT
        optimizer.step()

KeyError: 1931196